In [4]:
import json
from pprint import pprint


def isNumber(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

stats = {}
specFiles = ['data/EDM.json','data/BMM.json','data/BC.json',
             'data/PerR.json', 'data/TnT.json','data/TM.json']
for specFile in specFiles:
    jsonSpec = json.load(open(specFile))
    stats[jsonSpec['specificationName']] = jsonSpec

Subject Systems
============

In [2]:
import csv
from collections import Counter
with open('subjectSystemsTable.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    header = ['noClafer','noConstraint', 'noAbsClafer','noIntSetReference','noSetReference','noUnboundedClaferMult',
              'noUnobundedGroupMult', 'description']
    writer.writerow(['system']+header)
    for spec in stats:
        row = []
        row.append(spec.replace('_', ' '))
        for field in header:
            if field == 'description':
                row.append('TODO')
            else:
                row.append(stats[spec]['originalStats'][field])
        writer.writerow(row)
    

RQ1: Efficiency & Effectiveness
==========

In [7]:
import csv
from collections import Counter
from collections import defaultdict

with open('evalTable.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    modelHeader = ['noClaferOrig','noConstraintOrig', 'noClaferBase','noConstraintBase',
              'noClaferFlat','noConstraintFlat','noIlpDecVar','noIlpConstraint']
    compHeader = ['alloy', 'choco', 'bound']
    anomalyHeader = ['DEAD_CLAFER', 'UNBOUNDED_CLAFER', 'FALSE_LB_CMULT', 'FALSE_UB_CMULT', 'FALSE_UNBOUNDED_CMULT',
            'FALSE_LB_GMULT', 'FALSE_UB_GMULT', 'FALSE_UNBOUNDED_GMULT',
              'LOWER_BOUNDED_ATTR', 'UPPER_BOUNDED_ATTR', 'UNBOUNDED_ATTR']
    writer.writerow(['system'] + modelHeader + compHeader + [ n.replace('_', ' ') for n in anomalyHeader ])
    
    compData = {}
    with open('compEffortData.csv', 'r') as fp:
        reader = csv.reader(fp, delimiter=',', quotechar='"')
        # next(reader, None)  # skip the headers
        for row in reader:
            if row[0] != 'system':
                compData[row[0]] = {}
                compData[row[0]]['alloy'] = row[1]
                compData[row[0]]['choco'] = row[2]
                compData[row[0]]['bound'] = row[3]
    allOrigNoClafer = 0
    allOrigNoConstraint = 0
    allDropNoClafer = 0
    allDropNoConstraint = 0
    allFlatNoClafer = 0
    allFlatNoConstraint = 0
    allDecVar = 0
    allIlpCstr = 0
    allComp =  {field: 0 for field in compHeader }
    allAnomaly = {field: 0 for field in anomalyHeader}
    validFieldCount = defaultdict(int)
    
    for spec in stats:
        anomalies = []
        for i in range(0,len(stats[spec]['analysisResults'])):
            anomalies.append(stats[spec]['analysisResults'][i]['anomalyTypes'])
        counter = Counter([item for sublist in anomalies for item in sublist])
        row = []
        row.append(spec.replace('_', ' '))
        allOrigNoClafer += stats[spec]['originalStats']['noClafer']
        allOrigNoConstraint += stats[spec]['originalStats']['noConstraint']
        allDropNoClafer += stats[spec]['afterConstraintDropStats']['noClafer']
        allDropNoConstraint += stats[spec]['afterConstraintDropStats']['noConstraint']
        allFlatNoClafer += stats[spec]['afterFlattenigStats']['noClafer']
        allFlatNoConstraint += stats[spec]['afterFlattenigStats']['noConstraint']
        allDecVar += stats[spec]['analysisResults'][1]['lbIlpRes']['model']['noDecVars']
        allIlpCstr += stats[spec]['analysisResults'][1]['lbIlpRes']['model']['noConstraints']
        
        row.append(stats[spec]['originalStats']['noClafer'])
        row.append(stats[spec]['originalStats']['noConstraint'])
        row.append(stats[spec]['afterConstraintDropStats']['noClafer'])
        row.append(stats[spec]['afterConstraintDropStats']['noConstraint'])
        row.append(stats[spec]['afterFlattenigStats']['noClafer'])
        row.append(stats[spec]['afterFlattenigStats']['noConstraint'])
        row.append(stats[spec]['analysisResults'][1]['lbIlpRes']['model']['noDecVars'])
        row.append(stats[spec]['analysisResults'][1]['lbIlpRes']['model']['noConstraints'])
        for field in compHeader:
            val = compData[spec][field]
            if isNumber(val):
                allComp[field] = allComp[field] + float(val)
                validFieldCount[field] += 1
            row.append(val)
        for field in anomalyHeader:
            allAnomaly[field] = counter[field] + allAnomaly[field]
            row.append(counter[field])
        writer.writerow(row)
    row = []
    row.append('All')
    row.append(allOrigNoClafer)
    row.append(allOrigNoConstraint)
    row.append(allDropNoClafer)
    row.append(allDropNoConstraint)
    row.append(allFlatNoClafer)
    row.append(allFlatNoConstraint)
    row.append(allDecVar)
    row.append(allIlpCstr)
    for i in compHeader:
        row.append(round(allComp[i]/validFieldCount[i],4))
    for i in anomalyHeader:
        row.append(allAnomaly[i])
    writer.writerow(row)